### SMART ATTENDANCE SYSTEM USING FACE RECOGNITION
### TEAM MEMBERS: AKHILESH YADAV,ANKUSH VERMA,BHASHKARJYA NATH,RITIK AGRAWAL

In [1]:
'''
    This shell is used for including all 
    the libraries we need for project.
'''
import os
import csv
import cv2
import time
import numpy as np
from PIL import Image
from threading import Thread
import datetime
import pandas as pd

In [2]:
'''
    This function is written to check whether
    Cam is working properly or not.
'''

def check_cam():
    #cascade_face contain xml file for recognising face
    cascade_face = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    cam=cv2.VideoCapture(0)
    try:
        while True:
            ret,img=cam.read()
            if ret==False:
                break
            #this will convert the img into gray scale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = cascade_face.detectMultiScale(gray, 1.3, 5,minSize=(30, 30),flags = cv2.CASCADE_SCALE_IMAGE)
            for a,b,c,d in faces:
                #cv2.rectanlge(img_soucre,StartPoint,EndPoint,RectangleColor,Thickness) will 
                #generate rectangle around detected faces
                cv2.rectangle(img, (a, b), (a + c, b + d), (255,255,255), 2)

            #display
            cv2.imshow("Display",img)
            #enter q for closing display window
            if cv2.waitKey(1) & 0xFF==ord('q'):
                break
        #It is necessary to release cam before exiting the function
        #cam.release releases the cam
        cam.release()
        cv2.destroyAllWindows()
    except:
        cam.release()
        cv2.destroyAllWindows()

In [3]:
'''
    This function is written to capture images
    for Training of the model.This create a 
    Dataset of image with the name of the person.
'''
def capture_image():
    Id=int(input("Please Enter your Id: "))
    Name=input("Please Enter Your Name: ")
    cascade_face = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    cam = cv2.VideoCapture(0)
    img_cnt = 0
        
    while True:
        ret,img=cam.read()
        if ret==False:
            break
        #this will convert the img into gray scale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = cascade_face.detectMultiScale(gray, 1.3, 5,minSize=(30, 30),flags = cv2.CASCADE_SCALE_IMAGE)
        for a,b,c,d in faces:
            #cv2.rectanlge(img_soucre,StartPoint,EndPoint,RectangleColor,Thickness) will 
            #generate rectangle around detected faces
                
            cv2.rectangle(img, (a, b), (a + c, b + d), (255,255,255), 2)
            #display
            cv2.imshow("Display",img)
            cv2.imwrite("TrainingImage" + os.sep +Name + "."+str(Id) + '.' +
                            str(img_cnt) + ".jpg", gray[b:b+d, a:a+c])
                
        img_cnt=img_cnt+1
        #enter q for closing display window
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
#         elif img_cnt>100:
#             print("Enough Sample pics Taken!!!")
#             break
    #It is necessary to release cam before exiting the function
    #cam.release releases the cam
    cam.release()
    cv2.destroyAllWindows()
    #Writing Student Details in csv File
    row = [Id, Name]
    with open("StudentDetails"+os.sep+"StudentDetails.csv", 'a+') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(row)
    csvFile.close()

In [4]:
'''
    These Function are written for Training the 
    model.counter_img function counts the number 
    of images used for training.getImagesAndLabels
    function return List of images and corresponding 
    Id.
'''

def getImagesAndLabels(path):
    #path of all the files in the folder Training Image
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    faces = []
    Ids = []
    for imagePath in imagePaths:
        pilImage = Image.open(imagePath).convert('L')
        imageNp = np.array(pilImage, 'uint8')
        #Extracting Id from img names
        Id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces.append(imageNp)
        Ids.append(Id)
    return faces, Ids

def counter_img(path):
    imgcounter = 1
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    for imagePath in imagePaths:
        print(str(imgcounter) + " Images Trained", end="\r")
        time.sleep(0.008)
        imgcounter += 1

def train_model():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    detector = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    faces, Id = getImagesAndLabels("TrainingImage")
    Thread(target = recognizer.train(faces, np.array(Id))).start()
    Thread(target = counter_img("TrainingImage")).start()
    recognizer.save("TrainingImageLabel"+os.sep+"Trainner.yml")
    print("All")

In [5]:
def use_model():
    recognizer = cv2.face.LBPHFaceRecognizer_create()  
    recognizer.read("TrainingImageLabel"+os.sep+"Trainner.yml")
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    df = pd.read_csv("StudentDetails"+os.sep+"StudentDetails.csv")
    font = cv2.FONT_HERSHEY_SIMPLEX
    df.columns = ['ID', 'NAME']
    col_names=['ID','NAME','DATE','TIME']
    attendance = pd.DataFrame(columns=col_names)

    # start realtime video capture
    cam = cv2.VideoCapture(0)
    cam.set(3, 640) 
    cam.set(4, 480) 
    minW = 0.1 * cam.get(3)
    minH = 0.1 * cam.get(4)

    while True:
        ret, im = cam.read()
        if ret==False:
            break
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, 1.2, 5,
                minSize = (int(minW), int(minH)),flags = cv2.CASCADE_SCALE_IMAGE)
        for(x, y, w, h) in faces:
            cv2.rectangle(im, (x, y), (x+w, y+h), (255,255,255), 1)
            Id, conf = recognizer.predict(gray[y:y+h, x:x+w])
            if conf < 100:
                aa = df.loc[df['ID'] == Id]['NAME'].values
                tt = str(Id)+"-"+aa
            else:
                Id = 'Unknown'
                tt = str(Id)

            if (100-conf) > 67:
                ts = time.time()
                date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
                timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                aa = str(aa)[2:-2]
                attendance.loc[len(attendance)] = [Id, aa, date, timeStamp]

            tt = str(tt)[2:-2]
            if(100-conf) > 67:
                cv2.putText(im, " [Pass]", (x+10,y-5), font, .5, (255, 255, 255), 1)
            
            cv2.putText(im, str(tt), (x, y+h+10), font, .5, (255, 255, 255), 1)

        attendance = attendance.drop_duplicates(subset=['ID'], keep='first')
        cv2.imshow('Attendance', im)
        if (cv2.waitKey(1) == ord('q')):
            break
    ts = time.time()
    date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
    timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
    Hour, Minute, Second = timeStamp.split(":")
    fileName = "Attendance"+os.sep+"Attendance_"+date+"_"+Hour+"-"+Minute+".csv"
    attendance.to_csv(fileName, index=False)
    print("Attendance Successful")
    cam.release()
    cv2.destroyAllWindows()

In [6]:
def main_menu():
    print(10*("*"),"MAIN MENU",10*("*"),"\n",end="")
    print("[1] Check Your Camera")
    print("[2] Capture Image for Model Training")
    print("[3] Train Your Model")
    print("[4] Use Your Model")
    print("[5] For Exit")
    choice=int(input("Enter Your Choice"))
    if choice==1:
        check_cam()
        return False
    elif choice==2:
        capture_image()
        return False
    elif choice==3:
        train_model()
        return False
    elif choice==4:
        use_model()
        return False
    elif choice==5:
        print("Thank you")
        return True
    else:
        print("Invalid Choice: ")
        return False        
        

if __name__=="__main__":
    while True:
        ret=main_menu()
        if ret:
            break
    exit

********** MAIN MENU ********** 
[1] Check Your Camera
[2] Capture Image for Model Training
[3] Train Your Model
[4] Use Your Model
[5] For Exit
Enter Your Choice1
********** MAIN MENU ********** 
[1] Check Your Camera
[2] Capture Image for Model Training
[3] Train Your Model
[4] Use Your Model
[5] For Exit
Enter Your Choice2
Please Enter your Id: 187210
Please Enter Your Name: Bhashkarjya
********** MAIN MENU ********** 
[1] Check Your Camera
[2] Capture Image for Model Training
[3] Train Your Model
[4] Use Your Model
[5] For Exit
Enter Your Choice3
All Images Trained
********** MAIN MENU ********** 
[1] Check Your Camera
[2] Capture Image for Model Training
[3] Train Your Model
[4] Use Your Model
[5] For Exit
Enter Your Choice4
Attendance Successful
********** MAIN MENU ********** 
[1] Check Your Camera
[2] Capture Image for Model Training
[3] Train Your Model
[4] Use Your Model
[5] For Exit
Enter Your Choice5
Thank you
